In [1]:
from flask import Flask, request
import json
from textblob import TextBlob

import pandas as pd

In [82]:
inverted_index = {}
forward_index = []
documents_id = []

In [168]:
app = Flask(__name__)

@app.route("/")
def hello():
    return "Hello World from Scissors!"


@app.route('/about')
def about():
    return 'The about page'
        

@app.route('/ranging', methods=["POST"])
def ranging():
    """
    Returns:
    List with dictionaries according to ranged documents ID as json file
    """
    list_of_dicts = request.json 
    documents = [Document(**doc) for doc in list_of_dicts]
    return json.dumps([doc.to_dict() for doc in documents])
    
    
@app.route('/snippets', methods=["POST"])
def get_snippets():
    """
    Returns:   
    """     
    list_of_dicts = request.json 
    
    documents = [Document(**doc) for doc in list_of_dicts]
    result = [doc.to_dict() for doc in documents]
    
    return json.dumps(result)


@app.route('/return_page', methods=["POST"])
def return_page():
    """
    Returns:
    
    """
    list_of_dicts = request.json 
    documents = [Document(**doc) for doc in list_of_dicts]
    result = [doc.to_dict() for doc in documents]
    
    return "html"



@app.route('/indexator',  methods=["POST"])
def add_to_index():
    """
    Returns:
    
    """
    document = request.json 
    add_inverted_index(document)
    add_forward_index(document)
   
    return "added to index"


@app.route("/search", methods=["POST"])
def search():
    search_words = request.json["search_word"]
    
    result = search_one(search_words)
    
    return json.dumps(result)


@app.route('/snippets', methods=["POST"])
def get_snippets():
    """
    Returns:   
    """     
    search_words = request.json["search_word"]
    result = search_one(search_words)
    dicts = []
    for doc_id in result:
        val = list(df[df.id==doc_id].text.values)
        dicts.append({str(doc_id): val})
    return json.dumps(dicts[0])

In [169]:
if __name__ == "__main__":
    app.run(host='0.0.0.0', port=13502)

 * Running on http://0.0.0.0:13502/ (Press CTRL+C to quit)
127.0.0.1 - - [08/Nov/2018 18:59:27] "POST /search HTTP/1.1" 200 -
127.0.0.1 - - [08/Nov/2018 18:59:42] "POST /search HTTP/1.1" 200 -
127.0.0.1 - - [08/Nov/2018 18:59:49] "POST /search HTTP/1.1" 200 -
127.0.0.1 - - [08/Nov/2018 19:00:00] "POST /search HTTP/1.1" 200 -


In [108]:
def add_inverted_index(document):
    tokens = TextBlob(document["text_searchable"]).words
    
    if document["id"] not in documents_id:
        documents_id.append(document["id"])
        for token in tokens:
            if token in inverted_index.keys():
                inverted_index[token].append({"doc_id": int(document["id"])})
            else:
                inverted_index[token] = [{"doc_id": int(document["id"])}]
          

In [109]:
def add_forward_index(document):
    if document["id"] not in documents_id:
        documents_id.append(document["id"])
        forward_index.append({"doc_id": int(document["id"]), "text": document["text"]})

In [167]:
def search_one(words):
    idx = []
    words = words.split(" ")
      
    for word in words:
        if word in inverted_index.keys():
            idx.append(set( [doc["doc_id"] for doc in inverted_index[word]]))     
    
    indexes = idx[0]
    for doc_set in idx:
        indexes = indexes.intersection(doc_set)
    
    return list(indexes)

In [ ]:
def get_snippets():
    """
    Returns:   
    """     
    search_words = request.json["search_word"]
    result = search_one(search_words)
    
    dicts = []
    for doc_id in result:
        val = str(df[df.id==doc_id].text.values)
        dicts.append({str(doc_id): val})
    return json.dumps(dicts[0])

In [ ]:
fd

In [78]:
df = pd.read_csv("../../../Data/eval_texts.csv", sep='\t')

In [86]:
%%time
for i in range(len(df)):
    doc = df.loc[i].to_dict()
    add_inverted_index(doc)
    add_forward_index(doc)

CPU times: user 1min 27s, sys: 562 ms, total: 1min 28s
Wall time: 1min 28s


In [87]:
print(len(inverted_index))
print(len(forward_index))

20563
0


In [22]:
inverted_index.keys()

dict_keys(['должност', 'обязанност', 'грамот', 'выклад', 'товар', 'чиртон', 'контрол', 'ценник', 'срок', 'годност', 'размещен', 'реклам', 'материал', 'предоставлен', 'фотоотчет', 'по', 'эл', 'почт', 'whatsapp', 'требован', 'опыт', 'работ', 'приветств', 'налич', 'л/а', 'предпочтител', 'кандидат', 'без', 'всем', 'с', 'нул', 'обуч', 'водительск', 'прав', 'категори', 'b', 'мы', 'предлага', 'официал', 'трудоустройств', 'гсм', 'входит', 'в', 'заработ', 'плат', 'полный', 'рабоч', 'ден', 'разъезд', 'характер', 'компан', 'требуется', 'курьер', 'В', 'молод', 'друж', 'коллектив', 'требуются', 'добросовест', 'сотрудник', 'для', 'разнос', 'газет', 'почтов', 'ящик', 'жил', 'дом', 'выезд', 'командиров', 'зарплат', 'сдел', 'от', '8000', 'руб', 'недел', 'выплат', 'еженедел', 'задерж', 'и', 'проезд', 'оплачива', 'полна', 'занятост', '8', '00', 'до', '18', 'понедельник', 'пятниц', 'не', 'ука', 'ез', 'на', 'встреч', 'клиент', 'вруч', 'продукци', 'бан', 'прово', 'консультаци', 'продукт', 'ответствен', 'обр

In [18]:
df.loc[:4]

,uid,id,text,text_searchable,text_hash64,text_lang
0,-9223023784164508486,394513,Должностные обязанности: * Грамотная выкладка ...,должност обязанност грамот выклад товар чиртон...,4618598447649122478,ru
1,-9222980755089285550,8870790,В компанию требуется Курьер В молодой дружный ...,в компан требуется курьер В молод друж коллект...,-1436805725984275795,ru
2,-9222464969325259240,1866394,Должностные обязанности: • Ездить на встречу с...,должност обязанност ез на встреч с клиент вруч...,-2338421194536715427,ru
3,-9222236725167340775,6329712,Приглашаем на работу в компанию SushiBox ГДЕ П...,приглаша на работ в компан sushibox где платят...,-559923134811030623,ru
4,-9220847903682959224,7536361,Требования: В организацию по доставке питания ...,требован в организаци по достав питан услов оп...,-2668564901270524930,ru
